# 📚 Word Embeddings: Word2Vec and GloVe Tutorial

This notebook provides a comprehensive guide to understanding and implementing **Word2Vec** and **GloVe** word embeddings.

## Table of Contents
1. Introduction to Word Embeddings
2. Word2Vec: Theory and Implementation
   - Skip-gram Model
   - CBOW Model
3. GloVe: Theory and Implementation
4. Comparing Word2Vec and GloVe
5. Practical Applications

---

## 1. Setup and Installation

In [ ]:
# Install required libraries
!pip install gensim==4.3.2 nltk matplotlib numpy scikit-learn --quiet

# Download NLTK data
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('brown')

print("✅ Setup complete!")

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from gensim.models import Word2Vec, KeyedVectors
from nltk.corpus import brown, stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import warnings
warnings.filterwarnings('ignore')

print(f"Gensim version: {gensim.__version__}")

---
## 2. Introduction to Word Embeddings

### Why Word Embeddings?

Traditional text representations like **One-Hot Encoding** and **Bag of Words** have limitations:
- High dimensionality (vocabulary size)
- No semantic meaning captured
- No relationship between words

**Word Embeddings** solve these problems by:
- Representing words as dense vectors (typically 50-300 dimensions)
- Capturing semantic relationships
- Similar words have similar vectors

In [ ]:
# Demonstration: One-Hot Encoding vs Word Embeddings

vocabulary = ["king", "queen", "man", "woman", "child"]

# One-Hot Encoding
print("=" * 50)
print("ONE-HOT ENCODING")
print("=" * 50)
for i, word in enumerate(vocabulary):
    one_hot = [0] * len(vocabulary)
    one_hot[i] = 1
    print(f"{word:8} -> {one_hot}")

print("\n❌ Problems:")
print("   - All words are equally distant")
print("   - No semantic meaning")
print("   - Sparse vectors (mostly zeros)")

# Word Embeddings (hypothetical example)
print("\n" + "=" * 50)
print("WORD EMBEDDINGS (Dense Vectors)")
print("=" * 50)
embeddings = {
    "king":   [0.8, 0.6, 0.9, 0.1],
    "queen":  [0.7, 0.6, 0.1, 0.9],
    "man":    [0.5, 0.4, 0.8, 0.2],
    "woman":  [0.4, 0.4, 0.2, 0.8],
    "child":  [0.3, 0.2, 0.5, 0.5]
}
for word, vec in embeddings.items():
    print(f"{word:8} -> {vec}")

print("\n✅ Benefits:")
print("   - Dense, low-dimensional")
print("   - Similar words have similar vectors")
print("   - Captures semantic relationships")

---
## 3. Word2Vec: Theory

**Word2Vec** was developed by Mikolov et al. at Google (2013). It learns word embeddings by predicting context.

### Key Idea: "You shall know a word by the company it keeps" - J.R. Firth

### Two Architectures:

#### 1. Skip-gram
- **Input**: Center word
- **Output**: Context words
- Better for rare words
- Slower training

#### 2. CBOW (Continuous Bag of Words)
- **Input**: Context words
- **Output**: Center word
- Better for frequent words
- Faster training

In [ ]:
# Visualize Skip-gram vs CBOW

sentence = "The quick brown fox jumps over the lazy dog"
words = sentence.lower().split()
window_size = 2

print("Sentence:", sentence)
print(f"Window size: {window_size}")
print("\n" + "=" * 60)

# Skip-gram examples
print("\n📘 SKIP-GRAM: Predict context from center word")
print("-" * 60)
center_idx = 4  # "jumps"
center_word = words[center_idx]
context = words[max(0, center_idx-window_size):center_idx] + \
          words[center_idx+1:center_idx+window_size+1]

print(f"Center word: '{center_word}'")
print(f"Context words: {context}")
print(f"\nTraining pairs (input -> output):")
for ctx in context:
    print(f"   {center_word} -> {ctx}")

# CBOW examples
print("\n📗 CBOW: Predict center from context words")
print("-" * 60)
print(f"Context words: {context}")
print(f"Target word: '{center_word}'")
print(f"\nTraining pair:")
print(f"   {context} -> {center_word}")

### Word2Vec Architecture Diagram

In [ ]:
# Visual representation of Word2Vec architecture

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Skip-gram
ax1 = axes[0]
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.set_aspect('equal')
ax1.axis('off')
ax1.set_title('Skip-gram Model', fontsize=14, fontweight='bold')

# Input layer
ax1.add_patch(plt.Rectangle((1, 4), 2, 2, fill=True, color='lightblue', ec='black'))
ax1.text(2, 5, 'Input\n(Center)', ha='center', va='center', fontsize=10)
ax1.text(2, 3.3, '"jumps"', ha='center', va='center', fontsize=9, style='italic')

# Hidden layer
ax1.add_patch(plt.Rectangle((4, 4), 2, 2, fill=True, color='lightgreen', ec='black'))
ax1.text(5, 5, 'Hidden\nLayer', ha='center', va='center', fontsize=10)
ax1.text(5, 3.3, 'Word Vector', ha='center', va='center', fontsize=9)

# Output layer
for i, word in enumerate(['brown', 'fox', 'over', 'the']):
    y = 7.5 - i * 2
    ax1.add_patch(plt.Rectangle((7, y-0.4), 2, 0.8, fill=True, color='lightyellow', ec='black'))
    ax1.text(8, y, f'"{word}"', ha='center', va='center', fontsize=9)

# Arrows
ax1.annotate('', xy=(4, 5), xytext=(3, 5), arrowprops=dict(arrowstyle='->', color='black'))
for i in range(4):
    y = 7.5 - i * 2
    ax1.annotate('', xy=(7, y), xytext=(6, 5), arrowprops=dict(arrowstyle='->', color='black', alpha=0.5))

# CBOW
ax2 = axes[1]
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 10)
ax2.set_aspect('equal')
ax2.axis('off')
ax2.set_title('CBOW Model', fontsize=14, fontweight='bold')

# Input layer (context words)
for i, word in enumerate(['brown', 'fox', 'over', 'the']):
    y = 7.5 - i * 2
    ax2.add_patch(plt.Rectangle((1, y-0.4), 2, 0.8, fill=True, color='lightblue', ec='black'))
    ax2.text(2, y, f'"{word}"', ha='center', va='center', fontsize=9)

# Hidden layer
ax2.add_patch(plt.Rectangle((4, 4), 2, 2, fill=True, color='lightgreen', ec='black'))
ax2.text(5, 5, 'Hidden\n(Average)', ha='center', va='center', fontsize=10)

# Output layer
ax2.add_patch(plt.Rectangle((7, 4), 2, 2, fill=True, color='lightyellow', ec='black'))
ax2.text(8, 5, 'Output\n(Target)', ha='center', va='center', fontsize=10)
ax2.text(8, 3.3, '"jumps"', ha='center', va='center', fontsize=9, style='italic')

# Arrows
for i in range(4):
    y = 7.5 - i * 2
    ax2.annotate('', xy=(4, 5), xytext=(3, y), arrowprops=dict(arrowstyle='->', color='black', alpha=0.5))
ax2.annotate('', xy=(7, 5), xytext=(6, 5), arrowprops=dict(arrowstyle='->', color='black'))

plt.tight_layout()
plt.savefig('word2vec_architecture.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n💡 Key Difference:")
print("   Skip-gram: 1 center word → multiple context words")
print("   CBOW: multiple context words → 1 center word")

---
## 4. Training Word2Vec from Scratch (Simplified)

Let's implement a simplified version to understand the mechanics.

In [ ]:
class SimpleWord2Vec:
    """
    Simplified Word2Vec implementation for educational purposes.
    Uses Skip-gram with negative sampling.
    """

    def __init__(self, sentences, embedding_dim=50, window_size=2, learning_rate=0.01):
        self.embedding_dim = embedding_dim
        self.window_size = window_size
        self.learning_rate = learning_rate

        # Build vocabulary
        self.word_counts = Counter(word for sent in sentences for word in sent)
        self.vocab = list(self.word_counts.keys())
        self.vocab_size = len(self.vocab)

        # Word to index mapping
        self.word2idx = {word: idx for idx, word in enumerate(self.vocab)}
        self.idx2word = {idx: word for word, idx in self.word2idx.items()}

        # Initialize embeddings randomly
        self.W_input = np.random.randn(self.vocab_size, embedding_dim) * 0.01
        self.W_output = np.random.randn(embedding_dim, self.vocab_size) * 0.01

        print(f"Vocabulary size: {self.vocab_size}")
        print(f"Embedding dimension: {embedding_dim}")

    def generate_training_data(self, sentences):
        """Generate (center, context) pairs for Skip-gram."""
        training_data = []
        for sentence in sentences:
            for i, center_word in enumerate(sentence):
                # Get context words within window
                start = max(0, i - self.window_size)
                end = min(len(sentence), i + self.window_size + 1)

                for j in range(start, end):
                    if i != j:
                        context_word = sentence[j]
                        training_data.append((center_word, context_word))

        return training_data

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    def train_step(self, center_word, context_word):
        """Single training step with gradient descent."""
        center_idx = self.word2idx[center_word]
        context_idx = self.word2idx[context_word]

        # Forward pass
        hidden = self.W_input[center_idx]  # (embedding_dim,)
        output = np.dot(hidden, self.W_output)  # (vocab_size,)
        probs = self.sigmoid(output)

        # Create target (1 for context word, 0 for others)
        target = np.zeros(self.vocab_size)
        target[context_idx] = 1

        # Backward pass
        error = probs - target

        # Update output weights
        self.W_output -= self.learning_rate * np.outer(hidden, error)

        # Update input weights
        grad_hidden = np.dot(self.W_output, error)
        self.W_input[center_idx] -= self.learning_rate * grad_hidden

        # Return loss
        loss = -np.log(probs[context_idx] + 1e-10)
        return loss

    def train(self, sentences, epochs=100):
        """Train the model."""
        training_data = self.generate_training_data(sentences)
        print(f"Training pairs: {len(training_data)}")

        losses = []
        for epoch in range(epochs):
            total_loss = 0
            np.random.shuffle(training_data)

            for center, context in training_data:
                loss = self.train_step(center, context)
                total_loss += loss

            avg_loss = total_loss / len(training_data)
            losses.append(avg_loss)

            if (epoch + 1) % 20 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

        return losses

    def get_embedding(self, word):
        """Get embedding vector for a word."""
        if word in self.word2idx:
            return self.W_input[self.word2idx[word]]
        return None

    def most_similar(self, word, top_n=5):
        """Find most similar words using cosine similarity."""
        if word not in self.word2idx:
            return []

        word_vec = self.get_embedding(word).reshape(1, -1)
        similarities = cosine_similarity(word_vec, self.W_input)[0]

        # Get top similar words (excluding the word itself)
        similar_idx = np.argsort(similarities)[::-1][1:top_n+1]
        return [(self.idx2word[idx], similarities[idx]) for idx in similar_idx]

In [ ]:
# Train our simple Word2Vec on sample data

# Sample corpus
sample_corpus = [
    "the king rules the kingdom",
    "the queen rules the kingdom",
    "the king and queen are royal",
    "man and woman are human",
    "the prince is the son of king",
    "the princess is the daughter of queen",
    "king is a man who rules",
    "queen is a woman who rules",
    "boy and girl are young",
    "man was once a boy",
    "woman was once a girl"
]

# Tokenize
sentences = [sent.lower().split() for sent in sample_corpus]
print("Sample sentences:")
for sent in sentences[:3]:
    print(f"  {sent}")

# Train model
print("\n" + "="*50)
print("Training Simple Word2Vec...")
print("="*50)
simple_w2v = SimpleWord2Vec(sentences, embedding_dim=20, window_size=2)
losses = simple_w2v.train(sentences, epochs=100)

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 4))
plt.plot(losses, 'b-', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Training Loss Over Time', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

# Test similar words
print("\n" + "="*50)
print("Finding Similar Words")
print("="*50)
for test_word in ['king', 'queen', 'man', 'woman']:
    similar = simple_w2v.most_similar(test_word, top_n=3)
    print(f"\n'{test_word}' is similar to:")
    for word, score in similar:
        print(f"   {word}: {score:.4f}")

---
## 5. Using Gensim's Word2Vec (Production-Ready)

Now let's use Gensim's optimized implementation with a larger corpus.

In [ ]:
# Load Brown corpus (1 million+ words)
print("Loading Brown corpus...")
brown_sentences = brown.sents()

# Preprocess: lowercase
processed_sentences = [[word.lower() for word in sent] for sent in brown_sentences]

print(f"Number of sentences: {len(processed_sentences):,}")
print(f"Sample sentence: {processed_sentences[0]}")

In [ ]:
# Train Word2Vec using Gensim

print("\n" + "="*60)
print("Training Word2Vec with Gensim (Skip-gram)")
print("="*60)

# Skip-gram model (sg=1)
w2v_skipgram = Word2Vec(
    sentences=processed_sentences,
    vector_size=100,      # Embedding dimension
    window=5,             # Context window size
    min_count=5,          # Ignore words with freq < 5
    workers=4,            # Parallel training threads
    sg=1,                 # 1 = Skip-gram, 0 = CBOW
    epochs=10             # Training iterations
)

print(f"\nVocabulary size: {len(w2v_skipgram.wv):,}")
print(f"Embedding dimension: {w2v_skipgram.wv.vector_size}")

# Train CBOW model for comparison
print("\n" + "="*60)
print("Training Word2Vec with Gensim (CBOW)")
print("="*60)

w2v_cbow = Word2Vec(
    sentences=processed_sentences,
    vector_size=100,
    window=5,
    min_count=5,
    workers=4,
    sg=0,                 # CBOW
    epochs=10
)

print(f"\nVocabulary size: {len(w2v_cbow.wv):,}")

In [ ]:
# Explore Word2Vec embeddings

print("="*60)
print("Exploring Word2Vec Embeddings")
print("="*60)

# 1. Similar words
print("\n📌 Most Similar Words (Skip-gram):")
test_words = ['king', 'woman', 'computer', 'money']

for word in test_words:
    if word in w2v_skipgram.wv:
        similar = w2v_skipgram.wv.most_similar(word, topn=5)
        print(f"\n'{word}':")
        for sim_word, score in similar:
            print(f"   {sim_word}: {score:.4f}")

In [ ]:
# 2. Word Analogies: king - man + woman = ?

print("\n" + "="*60)
print("Word Analogies")
print("="*60)

analogies = [
    ('king', 'man', 'woman'),      # king - man + woman = queen?
    ('paris', 'france', 'italy'),  # paris - france + italy = rome?
    ('good', 'better', 'bad'),     # good - better + bad = worse?
]

print("\nAnalogy: A is to B as C is to ?")
print("-" * 40)

for a, b, c in analogies:
    try:
        # Find: a - b + c = ?
        result = w2v_skipgram.wv.most_similar(
            positive=[a, c],
            negative=[b],
            topn=3
        )
        print(f"\n{a} - {b} + {c} = ?")
        for word, score in result:
            print(f"   {word}: {score:.4f}")
    except KeyError as e:
        print(f"\n{a} - {b} + {c}: Word not in vocabulary")

In [ ]:
# 3. Word similarity score

print("\n" + "="*60)
print("Word Similarity Scores")
print("="*60)

word_pairs = [
    ('man', 'woman'),
    ('king', 'queen'),
    ('good', 'bad'),
    ('car', 'automobile'),
    ('happy', 'sad')
]

print("\nCosine Similarity between word pairs:")
print("-" * 40)
for w1, w2 in word_pairs:
    try:
        sim = w2v_skipgram.wv.similarity(w1, w2)
        print(f"{w1:12} <-> {w2:12}: {sim:.4f}")
    except KeyError:
        print(f"{w1:12} <-> {w2:12}: Word not found")

In [ ]:
# 4. Odd one out

print("\n" + "="*60)
print("Find the Odd One Out")
print("="*60)

word_lists = [
    ['breakfast', 'lunch', 'dinner', 'computer'],
    ['man', 'woman', 'child', 'car'],
    ['red', 'blue', 'green', 'happy']
]

for words in word_lists:
    try:
        odd = w2v_skipgram.wv.doesnt_match(words)
        print(f"\nWords: {words}")
        print(f"Odd one out: '{odd}'")
    except KeyError:
        print(f"\nWords: {words} - Some words not in vocabulary")

---
## 6. GloVe: Theory

**GloVe (Global Vectors)** was developed by Pennington et al. at Stanford (2014).

### Key Differences from Word2Vec:

| Aspect | Word2Vec | GloVe |
|--------|----------|-------|
| Approach | Predictive (neural network) | Count-based + Predictive |
| Training | Local context windows | Global co-occurrence matrix |
| Objective | Predict context/center | Minimize reconstruction error |

### GloVe's Key Insight:

The ratio of co-occurrence probabilities encodes meaning:

$$\frac{P(k|ice)}{P(k|steam)} = \begin{cases} \text{large} & \text{if } k = \text{"solid"} \\ \text{small} & \text{if } k = \text{"gas"} \\ \approx 1 & \text{if } k = \text{"water"} \end{cases}$$

In [ ]:
# Demonstrate GloVe's co-occurrence concept

print("="*60)
print("GloVe: Co-occurrence Matrix Concept")
print("="*60)

sample_text = """
The king sat on the throne. The queen stood beside the king.
The throne was golden. The king wore a crown.
The queen also wore a crown. The crown was jeweled.
"""

# Tokenize and build co-occurrence matrix
words = sample_text.lower().split()
words = [w.strip('.,') for w in words if w.strip('.,')]

vocab = list(set(words))
vocab_size = len(vocab)
word2idx = {w: i for i, w in enumerate(vocab)}

# Build co-occurrence matrix (window=2)
cooccur = np.zeros((vocab_size, vocab_size))
window = 2

for i, word in enumerate(words):
    for j in range(max(0, i-window), min(len(words), i+window+1)):
        if i != j:
            cooccur[word2idx[word]][word2idx[words[j]]] += 1

# Display subset of co-occurrence matrix
display_words = ['king', 'queen', 'throne', 'crown', 'golden']
display_idx = [word2idx[w] for w in display_words if w in word2idx]
display_words = [vocab[i] for i in display_idx]

print("\nCo-occurrence Matrix (subset):")
print("-" * 50)

# Print header
header = "         " + " ".join([f"{w:>8}" for w in display_words])
print(header)

# Print rows
for i, word in enumerate(display_words):
    row = [cooccur[word2idx[word]][word2idx[w]] for w in display_words]
    row_str = f"{word:>8} " + " ".join([f"{int(v):>8}" for v in row])
    print(row_str)

print("\n💡 GloVe learns embeddings by factorizing this matrix!")

In [ ]:
# Simplified GloVe implementation

class SimpleGloVe:
    """
    Simplified GloVe implementation for educational purposes.
    """

    def __init__(self, sentences, embedding_dim=50, window_size=2, x_max=100, alpha=0.75):
        self.embedding_dim = embedding_dim
        self.window_size = window_size
        self.x_max = x_max
        self.alpha = alpha

        # Build vocabulary
        all_words = [word for sent in sentences for word in sent]
        self.word_counts = Counter(all_words)
        self.vocab = list(self.word_counts.keys())
        self.vocab_size = len(self.vocab)
        self.word2idx = {w: i for i, w in enumerate(self.vocab)}
        self.idx2word = {i: w for w, i in self.word2idx.items()}

        print(f"Vocabulary size: {self.vocab_size}")

        # Build co-occurrence matrix
        self.cooccur = self._build_cooccurrence(sentences)

        # Initialize embeddings
        self.W = np.random.randn(self.vocab_size, embedding_dim) * 0.01
        self.W_context = np.random.randn(self.vocab_size, embedding_dim) * 0.01
        self.b = np.zeros(self.vocab_size)
        self.b_context = np.zeros(self.vocab_size)

    def _build_cooccurrence(self, sentences):
        """Build co-occurrence matrix with distance weighting."""
        cooccur = defaultdict(float)

        for sentence in sentences:
            for i, center in enumerate(sentence):
                for j in range(max(0, i-self.window_size), min(len(sentence), i+self.window_size+1)):
                    if i != j:
                        context = sentence[j]
                        distance = abs(i - j)
                        # Weight by inverse distance
                        cooccur[(self.word2idx[center], self.word2idx[context])] += 1.0 / distance

        print(f"Non-zero co-occurrences: {len(cooccur):,}")
        return cooccur

    def weighting_function(self, x):
        """GloVe weighting function to reduce impact of very frequent co-occurrences."""
        if x < self.x_max:
            return (x / self.x_max) ** self.alpha
        return 1.0

    def train(self, epochs=50, learning_rate=0.05):
        """Train GloVe embeddings."""
        print(f"\nTraining GloVe for {epochs} epochs...")
        losses = []

        # Get training pairs
        pairs = list(self.cooccur.items())

        for epoch in range(epochs):
            total_loss = 0
            np.random.shuffle(pairs)

            for (i, j), x_ij in pairs:
                if x_ij == 0:
                    continue

                # Compute cost
                weight = self.weighting_function(x_ij)

                # Predicted log co-occurrence
                pred = np.dot(self.W[i], self.W_context[j]) + self.b[i] + self.b_context[j]

                # Difference from actual log co-occurrence
                diff = pred - np.log(x_ij)

                # Weighted squared error
                loss = weight * diff ** 2
                total_loss += loss

                # Gradients
                grad = weight * diff

                # Update embeddings
                self.W[i] -= learning_rate * grad * self.W_context[j]
                self.W_context[j] -= learning_rate * grad * self.W[i]
                self.b[i] -= learning_rate * grad
                self.b_context[j] -= learning_rate * grad

            avg_loss = total_loss / len(pairs)
            losses.append(avg_loss)

            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

        # Final embeddings: average of word and context vectors
        self.embeddings = (self.W + self.W_context) / 2
        return losses

    def get_embedding(self, word):
        if word in self.word2idx:
            return self.embeddings[self.word2idx[word]]
        return None

    def most_similar(self, word, top_n=5):
        if word not in self.word2idx:
            return []

        word_vec = self.get_embedding(word).reshape(1, -1)
        similarities = cosine_similarity(word_vec, self.embeddings)[0]

        similar_idx = np.argsort(similarities)[::-1][1:top_n+1]
        return [(self.idx2word[idx], similarities[idx]) for idx in similar_idx]

In [ ]:
# Train simplified GloVe

print("="*60)
print("Training Simplified GloVe")
print("="*60)

# Use same sample corpus
glove_model = SimpleGloVe(sentences, embedding_dim=20, window_size=2)
glove_losses = glove_model.train(epochs=100, learning_rate=0.05)

In [ ]:
# Plot GloVe training loss
plt.figure(figsize=(10, 4))
plt.plot(glove_losses, 'g-', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('GloVe Training Loss', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

# Find similar words
print("\n" + "="*50)
print("GloVe Similar Words")
print("="*50)
for test_word in ['king', 'queen', 'man', 'woman']:
    similar = glove_model.most_similar(test_word, top_n=3)
    print(f"\n'{test_word}' is similar to:")
    for word, score in similar:
        print(f"   {word}: {score:.4f}")

---
## 7. Using Pre-trained GloVe Embeddings

Let's download and use Stanford's pre-trained GloVe embeddings.

In [ ]:
# Download pre-trained GloVe embeddings (50d version for speed)

import os
import urllib.request
import zipfile

glove_url = "https://nlp.stanford.edu/data/glove.6B.zip"
glove_dir = "glove"
glove_file = "glove.6B.50d.txt"

# Create directory
os.makedirs(glove_dir, exist_ok=True)

# Check if file exists
glove_path = os.path.join(glove_dir, glove_file)

if not os.path.exists(glove_path):
    print("Downloading GloVe embeddings (this may take a few minutes)...")
    zip_path = os.path.join(glove_dir, "glove.6B.zip")

    # Download
    urllib.request.urlretrieve(glove_url, zip_path)

    # Extract only the 50d file
    print("Extracting...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extract(glove_file, glove_dir)

    # Clean up zip
    os.remove(zip_path)
    print("Done!")
else:
    print("GloVe embeddings already downloaded.")

In [ ]:
# Load GloVe embeddings

def load_glove(filepath):
    """Load GloVe embeddings from file."""
    embeddings = {}
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

print("Loading GloVe embeddings...")
glove_embeddings = load_glove(glove_path)
print(f"Loaded {len(glove_embeddings):,} word vectors")
print(f"Embedding dimension: {len(list(glove_embeddings.values())[0])}")

In [ ]:
# Helper functions for GloVe

def glove_similarity(word1, word2, embeddings):
    """Calculate cosine similarity between two words."""
    if word1 not in embeddings or word2 not in embeddings:
        return None
    vec1 = embeddings[word1].reshape(1, -1)
    vec2 = embeddings[word2].reshape(1, -1)
    return cosine_similarity(vec1, vec2)[0][0]

def glove_most_similar(word, embeddings, top_n=5):
    """Find most similar words using GloVe embeddings."""
    if word not in embeddings:
        return []

    word_vec = embeddings[word].reshape(1, -1)

    # Calculate similarities with all words
    similarities = []
    for w, vec in embeddings.items():
        if w != word:
            sim = cosine_similarity(word_vec, vec.reshape(1, -1))[0][0]
            similarities.append((w, sim))

    # Sort and return top N
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

def glove_analogy(a, b, c, embeddings, top_n=3):
    """Solve analogy: a is to b as c is to ?"""
    if a not in embeddings or b not in embeddings or c not in embeddings:
        return []

    # target = b - a + c
    target = embeddings[b] - embeddings[a] + embeddings[c]
    target = target.reshape(1, -1)

    # Find most similar
    exclude = {a, b, c}
    similarities = []
    for w, vec in embeddings.items():
        if w not in exclude:
            sim = cosine_similarity(target, vec.reshape(1, -1))[0][0]
            similarities.append((w, sim))

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

In [ ]:
# Test pre-trained GloVe

print("="*60)
print("Pre-trained GloVe Embeddings")
print("="*60)

# Similar words
print("\n📌 Most Similar Words:")
for word in ['king', 'computer', 'happy']:
    similar = glove_most_similar(word, glove_embeddings, top_n=5)
    print(f"\n'{word}':")
    for w, score in similar:
        print(f"   {w}: {score:.4f}")

In [ ]:
# Famous analogies with GloVe

print("\n" + "="*60)
print("Word Analogies with Pre-trained GloVe")
print("="*60)

analogies = [
    ('man', 'king', 'woman'),       # man:king :: woman:?
    ('paris', 'france', 'tokyo'),   # paris:france :: tokyo:?
    ('slow', 'slower', 'fast'),     # slow:slower :: fast:?
    ('good', 'best', 'bad'),        # good:best :: bad:?
]

print("\nAnalogy: A is to B as C is to ?")
print("-" * 50)

for a, b, c in analogies:
    result = glove_analogy(a, b, c, glove_embeddings, top_n=3)
    print(f"\n{a}:{b} :: {c}:?")
    for word, score in result:
        print(f"   {word}: {score:.4f}")

---
## 8. Comparing Word2Vec and GloVe

In [ ]:
# Convert our trained Word2Vec to dictionary for comparison
w2v_embeddings = {word: w2v_skipgram.wv[word] for word in w2v_skipgram.wv.key_to_index}

print("="*60)
print("Comparison: Word2Vec vs GloVe")
print("="*60)

# Compare similarities for common words
test_pairs = [
    ('king', 'queen'),
    ('man', 'woman'),
    ('good', 'bad'),
    ('happy', 'sad')
]

print("\nSimilarity Scores:")
print("-" * 60)
print(f"{'Word Pair':<20} {'Word2Vec':>15} {'GloVe':>15}")
print("-" * 60)

for w1, w2 in test_pairs:
    # Word2Vec similarity
    try:
        w2v_sim = w2v_skipgram.wv.similarity(w1, w2)
    except:
        w2v_sim = None

    # GloVe similarity
    glove_sim = glove_similarity(w1, w2, glove_embeddings)

    w2v_str = f"{w2v_sim:.4f}" if w2v_sim else "N/A"
    glove_str = f"{glove_sim:.4f}" if glove_sim else "N/A"

    print(f"{w1+' - '+w2:<20} {w2v_str:>15} {glove_str:>15}")

In [ ]:
# Theoretical comparison

print("\n" + "="*70)
print("Theoretical Comparison")
print("="*70)

comparison = """
┌─────────────────┬────────────────────────┬────────────────────────┐
│ Aspect          │ Word2Vec               │ GloVe                  │
├─────────────────┼────────────────────────┼────────────────────────┤
│ Approach        │ Predictive (NN)        │ Count-based + Matrix   │
│ Training Input  │ Local context windows  │ Global co-occurrence   │
│ Objective       │ Predict context/center │ Reconstruct log-counts │
│ Memory          │ Low (streaming)        │ High (full matrix)     │
│ Speed           │ Faster for large data  │ Faster for small vocab │
│ Rare Words      │ Better (Skip-gram)     │ Depends on frequency   │
│ Parallelization │ Limited                │ Easy                   │
│ Interpretability│ Less intuitive         │ More intuitive         │
└─────────────────┴────────────────────────┴────────────────────────┘
"""
print(comparison)

print("\n💡 When to use which:")
print("-" * 50)
print("• Word2Vec: Large corpus, streaming data, rare words important")
print("• GloVe: Fixed corpus, interpretability needed, parallel training")

---
## 9. Visualization: t-SNE

Let's visualize word embeddings in 2D using t-SNE.

In [ ]:
# Select words for visualization

word_groups = {
    'royalty': ['king', 'queen', 'prince', 'princess', 'throne', 'crown', 'royal'],
    'family': ['man', 'woman', 'boy', 'girl', 'father', 'mother', 'son', 'daughter'],
    'animals': ['dog', 'cat', 'horse', 'bird', 'fish', 'lion', 'tiger', 'elephant'],
    'numbers': ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight'],
    'countries': ['america', 'china', 'japan', 'france', 'germany', 'india', 'russia', 'brazil']
}

# Collect words that exist in GloVe
words_to_plot = []
colors = []
color_map = {'royalty': 'red', 'family': 'blue', 'animals': 'green',
             'numbers': 'orange', 'countries': 'purple'}

for group, words in word_groups.items():
    for word in words:
        if word in glove_embeddings:
            words_to_plot.append(word)
            colors.append(color_map[group])

print(f"Words to visualize: {len(words_to_plot)}")

In [ ]:
# Create embedding matrix and apply t-SNE

embedding_matrix = np.array([glove_embeddings[word] for word in words_to_plot])

print("Applying t-SNE (this may take a moment)...")
tsne = TSNE(n_components=2, random_state=42, perplexity=min(5, len(words_to_plot)-1))
embeddings_2d = tsne.fit_transform(embedding_matrix)
print("Done!")

In [ ]:
# Plot t-SNE visualization

plt.figure(figsize=(14, 10))

# Plot points
for i, word in enumerate(words_to_plot):
    x, y = embeddings_2d[i]
    plt.scatter(x, y, c=colors[i], s=100, alpha=0.7)
    plt.annotate(word, (x, y), fontsize=10, ha='center', va='bottom',
                 xytext=(0, 5), textcoords='offset points')

# Add legend
legend_elements = [plt.Line2D([0], [0], marker='o', color='w',
                               markerfacecolor=color, markersize=10, label=group)
                   for group, color in color_map.items()]
plt.legend(handles=legend_elements, loc='upper right', fontsize=10)

plt.title('t-SNE Visualization of GloVe Word Embeddings', fontsize=14)
plt.xlabel('Dimension 1', fontsize=12)
plt.ylabel('Dimension 2', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('tsne_glove.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n💡 Notice how semantically similar words cluster together!")

---
## 10. Practical Applications

In [ ]:
# Application 1: Sentence Similarity

def sentence_embedding(sentence, embeddings, method='average'):
    """Convert sentence to embedding by averaging word vectors."""
    words = sentence.lower().split()
    word_vecs = [embeddings[w] for w in words if w in embeddings]

    if not word_vecs:
        return None

    if method == 'average':
        return np.mean(word_vecs, axis=0)
    elif method == 'sum':
        return np.sum(word_vecs, axis=0)

def sentence_similarity(sent1, sent2, embeddings):
    """Calculate similarity between two sentences."""
    emb1 = sentence_embedding(sent1, embeddings)
    emb2 = sentence_embedding(sent2, embeddings)

    if emb1 is None or emb2 is None:
        return None

    return cosine_similarity(emb1.reshape(1, -1), emb2.reshape(1, -1))[0][0]

# Test sentence similarity
print("="*60)
print("Application: Sentence Similarity")
print("="*60)

sentence_pairs = [
    ("The cat sat on the mat", "A kitten was sitting on the rug"),
    ("I love programming in Python", "Python is my favorite language"),
    ("The weather is nice today", "Machine learning is interesting"),
]

print("\nSentence Pair Similarity:")
print("-" * 60)

for sent1, sent2 in sentence_pairs:
    sim = sentence_similarity(sent1, sent2, glove_embeddings)
    print(f"\nSentence 1: {sent1}")
    print(f"Sentence 2: {sent2}")
    print(f"Similarity: {sim:.4f}")

In [ ]:
# Application 2: Simple Text Classification with Embeddings

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Sample dataset
texts = [
    # Positive
    "I love this product it is amazing",
    "This is wonderful and fantastic",
    "Great experience highly recommend",
    "Excellent quality very happy",
    "Best purchase ever so satisfied",
    "Really good impressed with quality",
    # Negative
    "This is terrible waste of money",
    "Very disappointed not worth it",
    "Poor quality would not recommend",
    "Horrible experience very upset",
    "Bad product total disappointment",
    "Awful never buying again"
]

labels = [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]  # 1=positive, 0=negative

# Convert texts to embeddings
X = []
valid_labels = []
for text, label in zip(texts, labels):
    emb = sentence_embedding(text, glove_embeddings)
    if emb is not None:
        X.append(emb)
        valid_labels.append(label)

X = np.array(X)
y = np.array(valid_labels)

print("="*60)
print("Application: Sentiment Classification")
print("="*60)
print(f"\nDataset: {len(X)} samples")
print(f"Embedding shape: {X.shape}")

# Train classifier
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.2%}")

# Test on new sentences
print("\nPredictions on new sentences:")
print("-" * 50)
new_texts = [
    "This is absolutely wonderful",
    "I hate this terrible product",
    "Okay not bad could be better"
]

for text in new_texts:
    emb = sentence_embedding(text, glove_embeddings)
    if emb is not None:
        pred = clf.predict(emb.reshape(1, -1))[0]
        prob = clf.predict_proba(emb.reshape(1, -1))[0]
        sentiment = "Positive 😊" if pred == 1 else "Negative 😞"
        print(f"\n'{text}'")
        print(f"   Prediction: {sentiment} (confidence: {max(prob):.2%})")

---
## 11. Summary

### Key Takeaways:

1. **Word Embeddings** represent words as dense vectors capturing semantic meaning

2. **Word2Vec** (Google, 2013):
   - Uses neural network to predict context
   - Two variants: Skip-gram and CBOW
   - Works with local context windows

3. **GloVe** (Stanford, 2014):
   - Uses global co-occurrence statistics
   - Optimizes to reconstruct log co-occurrence counts
   - More interpretable training objective

4. **Both capture**:
   - Semantic similarity (king ≈ queen)
   - Analogical relationships (king - man + woman ≈ queen)

5. **Applications**:
   - Text classification
   - Sentiment analysis
   - Named entity recognition
   - Machine translation
   - Question answering

In [ ]:
# Final summary visualization

print("\n" + "="*70)
print("📊 SUMMARY: Word2Vec vs GloVe")
print("="*70)

summary = """
┌────────────────────────────────────────────────────────────────────┐
│                        WORD EMBEDDINGS                            │
│                                                                   │
│  Traditional (One-Hot, BoW)      →    Dense Embeddings            │
│  - Sparse, high-dimensional           - Dense, low-dimensional    │
│  - No semantic meaning                - Capture semantics         │
│  - Words independent                  - Similar words → similar   │
│                                         vectors                   │
├────────────────────────────────────────────────────────────────────┤
│                                                                   │
│  ┌──────────────────┐          ┌──────────────────┐               │
│  │    WORD2VEC      │          │      GLOVE       │               │
│  │   (Google 2013)  │          │  (Stanford 2014) │               │
│  │                  │          │                  │               │
│  │  "Predictive"    │          │  "Count-based"   │               │
│  │                  │          │                  │               │
│  │  Skip-gram:      │          │  Co-occurrence   │               │
│  │  center → context│          │  matrix + Matrix │               │
│  │                  │          │  factorization   │               │
│  │  CBOW:           │          │                  │               │
│  │  context → center│          │                  │               │
│  └──────────────────┘          └──────────────────┘               │
│                                                                   │
│  Both achieve: king - man + woman ≈ queen                         │
└────────────────────────────────────────────────────────────────────┘
"""
print(summary)

print("\n🎓 Next Steps:")
print("-" * 50)
print("1. Try training on your own corpus")
print("2. Experiment with different hyperparameters")
print("3. Explore FastText (handles out-of-vocabulary words)")
print("4. Learn about contextualized embeddings (BERT, ELMo)")
print("\n✅ Tutorial Complete!")

---
## 12. Exercises

Try these exercises to deepen your understanding:

1. **Train Word2Vec on a different corpus** (e.g., Wikipedia articles)
2. **Compare Skip-gram vs CBOW** on the same data
3. **Create a simple search engine** using sentence embeddings
4. **Visualize word clusters** for your domain-specific vocabulary
5. **Build a text classifier** for your own dataset

In [ ]:
# Exercise starter code

# Exercise 1: Train on custom data
# your_corpus = [
#     "Your sentence here",
#     "Another sentence",
#     ...
# ]
# your_sentences = [sent.lower().split() for sent in your_corpus]
# your_model = Word2Vec(your_sentences, vector_size=100, window=5, min_count=1, epochs=50)

print("💡 Uncomment and modify the code above to train on your own data!")